### This notebook is used to train GBRT models and write result

In [4]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import pandas as pd
import numpy as np
import matplotlib
matplotlib.rcParams['figure.figsize'] = (12, 6)
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
from functools import partial
from scipy.stats import linregress
probit = np.array([-1.28155, -0.84162, -0.52440, -0.25335])
xx = np.concatenate((probit, np.array([0]), -np.flip(probit, axis = -1)))

In [5]:
quant = 0.1
def xgb_quantile_eval(preds, dmatrix, quantile=quant):
    """
    Customized evaluational metric that equals
    to quantile regression loss (also known as
    pinball loss).
    Quantile regression is regression that
    estimates a specified quantile of target's
    distribution conditional on given features.
    @type preds: numpy.ndarray
    @type dmatrix: xgboost.DMatrix
    @type quantile: float
    @rtype: float
    """
    labels = dmatrix.get_label()
    return ('q{}_loss'.format(quantile),
            np.nanmean((preds >= labels) * (1 - quantile) * (preds - labels) +
                       (preds < labels) * quantile * (labels - preds)))


def xgb_quantile_obj(preds, dmatrix, quantile=quant):
    """
    Computes first-order derivative of quantile
    regression loss and a non-degenerate
    substitute for second-order derivative.
    Substitute is returned instead of zeros,
    because XGBoost requires non-zero
    second-order derivatives. See this page:
    https://github.com/dmlc/xgboost/issues/1825
    to see why it is possible to use this trick.
    However, be sure that hyperparameter named
    `max_delta_step` is small enough to satisfy:
    ```0.5 * max_delta_step <=
       min(quantile, 1 - quantile)```.
    @type preds: numpy.ndarray
    @type dmatrix: xgboost.DMatrix
    @type quantile: float
    @rtype: tuple(numpy.ndarray)
    """
    try:
        assert 0 <= quantile <= 1
    except AssertionError:
        raise ValueError("Quantile value must be float between 0 and 1.")

    labels = dmatrix.get_label()
    errors = preds - labels

    left_mask = errors < 0
    right_mask = errors > 0

    grad = -quantile * left_mask + (1 - quantile) * right_mask
    hess = np.ones_like(preds)

    return grad, hess


# Example of usage:
# bst = xgb.train(hyperparams, train, num_rounds,
#                 obj=xgb_quantile_obj, feval=xgb_quantile_eval)

#### region = region name, j is the number of current model (j=1,2,...,5 in the paper)

In [6]:
# ME NH VT RI SEMASS WCMASS NEMASSBOST
region = 'NEMASSBOST'
j=5

### Train and write result

In [7]:
df_train = pd.read_csv('Data/'+region+'_train.csv')
df_dev = pd.read_csv('Data/'+region+'_dev.csv')
df_dev2 = pd.read_csv('Data/'+region+'_dev2.csv')

predictors = ['Hour', 'Month', 'Day', 'Weekday',
              '1D', '2D', '3D', '4D', '5D', '6D', '7D', 'Weekend?', 
              'D2', 'D3', 'D4', 'D5','DD1', 'DD2', 'DD4', 'DD5',
              'W1', 'W2', 'W4', 'W5']
dtrain = xgb.DMatrix(df_train[predictors].values, label = df_train['DEMAND'].values)
dtest = xgb.DMatrix(df_dev[predictors].values, label = df_dev['DEMAND'].values)
dtest2 = xgb.DMatrix(df_dev2[predictors].values, label = df_dev2['DEMAND'].values)

param = {'eta': 200.,
         'max_depth': 5,
         'min_child_weight': 3,
         #'subsample': 0.4,
         #'colsample_bytree': 0.6,
         'max_delta_step': 0.1}

watchlist  = [(dtrain,'train'), (dtest,'eval')]
num_round = 10000

quantile_result = np.zeros((dtest2.num_row(),9))
for i in range(9):
    bst = xgb.train(param, 
                    dtrain, 
                    num_round, 
                    watchlist,
                    obj = partial(xgb_quantile_obj, quantile=(i+1)/10),
                    feval = partial(xgb_quantile_eval, quantile=(i+1)/10), 
                    early_stopping_rounds = 200)
    quantile_result[:, i] = bst.predict(dtest2, ntree_limit=bst.best_ntree_limit)

result = np.zeros((quantile_result.shape[0],2))
for i in range(quantile_result.shape[0]):
    slope, intercept, r_value, p_value, std_err = linregress(xx, quantile_result[i,:])
    result[i,0] = intercept
    result[i,1] = slope
    
df_result = pd.DataFrame(result, columns = ['mean', 'std'])
df_result.to_csv('results/XGB_'+region+'_result'+str(j)+'.csv', index = False)

[0]	train-rmse:2926.23	eval-rmse:3143.33	train-q0.1_loss:287.427	eval-q0.1_loss:311.647
Multiple eval metrics have been passed: 'eval-q0.1_loss' will be used for early stopping.

Will train until eval-q0.1_loss hasn't improved in 200 rounds.
[1]	train-rmse:2906.59	eval-rmse:3123.5	train-q0.1_loss:285.427	eval-q0.1_loss:309.647
[2]	train-rmse:2886.96	eval-rmse:3103.68	train-q0.1_loss:283.427	eval-q0.1_loss:307.647
[3]	train-rmse:2867.33	eval-rmse:3083.86	train-q0.1_loss:281.428	eval-q0.1_loss:305.647
[4]	train-rmse:2847.7	eval-rmse:3064.04	train-q0.1_loss:279.428	eval-q0.1_loss:303.648
[5]	train-rmse:2828.08	eval-rmse:3044.22	train-q0.1_loss:277.428	eval-q0.1_loss:301.648
[6]	train-rmse:2808.47	eval-rmse:3024.41	train-q0.1_loss:275.428	eval-q0.1_loss:299.648
[7]	train-rmse:2788.86	eval-rmse:3004.6	train-q0.1_loss:273.428	eval-q0.1_loss:297.648
[8]	train-rmse:2769.25	eval-rmse:2984.79	train-q0.1_loss:271.429	eval-q0.1_loss:295.648
[9]	train-rmse:2749.66	eval-rmse:2964.98	train-q0.1_loss:

[91]	train-rmse:1188.84	eval-rmse:1359.97	train-q0.1_loss:105.448	eval-q0.1_loss:129.668
[92]	train-rmse:1171.14	eval-rmse:1340.92	train-q0.1_loss:103.45	eval-q0.1_loss:127.668
[93]	train-rmse:1153.52	eval-rmse:1321.9	train-q0.1_loss:101.461	eval-q0.1_loss:125.668
[94]	train-rmse:1135.99	eval-rmse:1302.9	train-q0.1_loss:99.4869	eval-q0.1_loss:123.668
[95]	train-rmse:1118.54	eval-rmse:1283.93	train-q0.1_loss:97.5411	eval-q0.1_loss:121.669
[96]	train-rmse:1101.24	eval-rmse:1265	train-q0.1_loss:95.6296	eval-q0.1_loss:119.669
[97]	train-rmse:1084.01	eval-rmse:1246.18	train-q0.1_loss:93.7397	eval-q0.1_loss:117.686
[98]	train-rmse:1066.9	eval-rmse:1227.33	train-q0.1_loss:91.8818	eval-q0.1_loss:115.689
[99]	train-rmse:1049.87	eval-rmse:1208.6	train-q0.1_loss:90.0279	eval-q0.1_loss:113.707
[100]	train-rmse:1032.98	eval-rmse:1189.84	train-q0.1_loss:88.2024	eval-q0.1_loss:111.716
[101]	train-rmse:1016.18	eval-rmse:1171.14	train-q0.1_loss:86.3813	eval-q0.1_loss:109.73
[102]	train-rmse:999.532	eva

[183]	train-rmse:294.837	eval-rmse:322.885	train-q0.1_loss:19.8117	eval-q0.1_loss:33.1851
[184]	train-rmse:292.629	eval-rmse:317.974	train-q0.1_loss:19.6485	eval-q0.1_loss:32.8223
[185]	train-rmse:291.118	eval-rmse:316.187	train-q0.1_loss:19.5468	eval-q0.1_loss:32.7988
[186]	train-rmse:290.334	eval-rmse:315.948	train-q0.1_loss:19.4776	eval-q0.1_loss:32.7632
[187]	train-rmse:289.206	eval-rmse:315.495	train-q0.1_loss:19.4128	eval-q0.1_loss:32.7151
[188]	train-rmse:288.219	eval-rmse:314.844	train-q0.1_loss:19.349	eval-q0.1_loss:32.6376
[189]	train-rmse:287.119	eval-rmse:318.665	train-q0.1_loss:19.2707	eval-q0.1_loss:32.1446
[190]	train-rmse:285.247	eval-rmse:311.054	train-q0.1_loss:19.1133	eval-q0.1_loss:32.0774
[191]	train-rmse:283.563	eval-rmse:309.887	train-q0.1_loss:18.9862	eval-q0.1_loss:32.1564
[192]	train-rmse:281.661	eval-rmse:307.632	train-q0.1_loss:18.8557	eval-q0.1_loss:31.9774
[193]	train-rmse:280.641	eval-rmse:307.488	train-q0.1_loss:18.7749	eval-q0.1_loss:32.1839
[194]	train

[275]	train-rmse:240.121	eval-rmse:282.315	train-q0.1_loss:15.5807	eval-q0.1_loss:36.442
[276]	train-rmse:240.072	eval-rmse:282.661	train-q0.1_loss:15.5817	eval-q0.1_loss:36.2206
[277]	train-rmse:240.04	eval-rmse:282.746	train-q0.1_loss:15.5366	eval-q0.1_loss:36.4614
[278]	train-rmse:238.754	eval-rmse:281.009	train-q0.1_loss:15.5108	eval-q0.1_loss:36.4978
[279]	train-rmse:238.647	eval-rmse:281.212	train-q0.1_loss:15.4852	eval-q0.1_loss:36.2888
[280]	train-rmse:238.029	eval-rmse:281.351	train-q0.1_loss:15.4588	eval-q0.1_loss:36.5764
[281]	train-rmse:237.145	eval-rmse:281.429	train-q0.1_loss:15.3859	eval-q0.1_loss:36.1804
[282]	train-rmse:236.857	eval-rmse:280.501	train-q0.1_loss:15.3973	eval-q0.1_loss:36.2799
[283]	train-rmse:236.019	eval-rmse:279.988	train-q0.1_loss:15.341	eval-q0.1_loss:36.3239
[284]	train-rmse:235.763	eval-rmse:280.649	train-q0.1_loss:15.3397	eval-q0.1_loss:36.4135
[285]	train-rmse:235.051	eval-rmse:281.13	train-q0.1_loss:15.2597	eval-q0.1_loss:36.103
[286]	train-rms

[367]	train-rmse:208.167	eval-rmse:266.623	train-q0.1_loss:13.1586	eval-q0.1_loss:39.9821
[368]	train-rmse:207.766	eval-rmse:266.4	train-q0.1_loss:13.1554	eval-q0.1_loss:40.0157
[369]	train-rmse:207.884	eval-rmse:266.393	train-q0.1_loss:13.1432	eval-q0.1_loss:39.9387
[370]	train-rmse:207.741	eval-rmse:266.573	train-q0.1_loss:13.1432	eval-q0.1_loss:40.1161
[371]	train-rmse:207.767	eval-rmse:267.165	train-q0.1_loss:13.14	eval-q0.1_loss:40.0279
[372]	train-rmse:207.355	eval-rmse:266.47	train-q0.1_loss:13.1295	eval-q0.1_loss:40.3735
[373]	train-rmse:207.447	eval-rmse:266.351	train-q0.1_loss:13.1217	eval-q0.1_loss:39.8868
[374]	train-rmse:207.027	eval-rmse:265.655	train-q0.1_loss:13.1064	eval-q0.1_loss:40.1983
[375]	train-rmse:207.513	eval-rmse:267.103	train-q0.1_loss:13.079	eval-q0.1_loss:40.2022
[376]	train-rmse:206.744	eval-rmse:266.48	train-q0.1_loss:13.0779	eval-q0.1_loss:40.3005
[377]	train-rmse:206.518	eval-rmse:267.079	train-q0.1_loss:13.0402	eval-q0.1_loss:40.2835
[378]	train-rmse:

...

[357]	train-rmse:168.093	eval-rmse:225.667	train-q0.8_loss:30.0276	eval-q0.8_loss:81.497
[358]	train-rmse:168.019	eval-rmse:225.389	train-q0.8_loss:29.9332	eval-q0.8_loss:81.9022
[359]	train-rmse:167.891	eval-rmse:226.005	train-q0.8_loss:29.9945	eval-q0.8_loss:81.6341
[360]	train-rmse:167.868	eval-rmse:225.729	train-q0.8_loss:29.8701	eval-q0.8_loss:82.1323
[361]	train-rmse:167.612	eval-rmse:226.08	train-q0.8_loss:29.9127	eval-q0.8_loss:82.0013
[362]	train-rmse:167.839	eval-rmse:225.772	train-q0.8_loss:29.8356	eval-q0.8_loss:81.8782
[363]	train-rmse:167.348	eval-rmse:226.079	train-q0.8_loss:29.8349	eval-q0.8_loss:81.8752
[364]	train-rmse:167.41	eval-rmse:225.788	train-q0.8_loss:29.7769	eval-q0.8_loss:82.0767
[365]	train-rmse:167.437	eval-rmse:226.531	train-q0.8_loss:29.8013	eval-q0.8_loss:81.9323
[366]	train-rmse:167.38	eval-rmse:225.95	train-q0.8_loss:29.7275	eval-q0.8_loss:82.1426
[367]	train-rmse:167.289	eval-rmse:226.365	train-q0.8_loss:29.771	eval-q0.8_loss:81.824
[368]	train-rmse:

[42]	train-rmse:2107.05	eval-rmse:2313.1	train-q0.9_loss:1830.84	eval-q0.9_loss:2048.82
[43]	train-rmse:2087.75	eval-rmse:2293.42	train-q0.9_loss:1812.84	eval-q0.9_loss:2030.82
[44]	train-rmse:2068.46	eval-rmse:2273.75	train-q0.9_loss:1794.84	eval-q0.9_loss:2012.82
[45]	train-rmse:2049.18	eval-rmse:2254.08	train-q0.9_loss:1776.84	eval-q0.9_loss:1994.82
[46]	train-rmse:2029.92	eval-rmse:2234.42	train-q0.9_loss:1758.84	eval-q0.9_loss:1976.82
[47]	train-rmse:2010.67	eval-rmse:2214.76	train-q0.9_loss:1740.84	eval-q0.9_loss:1958.82
[48]	train-rmse:1991.44	eval-rmse:2195.11	train-q0.9_loss:1722.84	eval-q0.9_loss:1940.82
[49]	train-rmse:1972.22	eval-rmse:2175.46	train-q0.9_loss:1704.84	eval-q0.9_loss:1922.82
[50]	train-rmse:1953.02	eval-rmse:2155.82	train-q0.9_loss:1686.84	eval-q0.9_loss:1904.82
[51]	train-rmse:1933.84	eval-rmse:2136.19	train-q0.9_loss:1668.84	eval-q0.9_loss:1886.82
[52]	train-rmse:1914.67	eval-rmse:2116.57	train-q0.9_loss:1650.84	eval-q0.9_loss:1868.82
[53]	train-rmse:1895.5

[134]	train-rmse:538.879	eval-rmse:597.353	train-q0.9_loss:302.48	eval-q0.9_loss:434.944
[135]	train-rmse:527.789	eval-rmse:582.258	train-q0.9_loss:291.336	eval-q0.9_loss:421.311
[136]	train-rmse:516.863	eval-rmse:567.159	train-q0.9_loss:280.368	eval-q0.9_loss:407.42
[137]	train-rmse:506.235	eval-rmse:552.144	train-q0.9_loss:269.642	eval-q0.9_loss:394.142
[138]	train-rmse:495.907	eval-rmse:537.58	train-q0.9_loss:259.115	eval-q0.9_loss:380.865
[139]	train-rmse:485.983	eval-rmse:523.175	train-q0.9_loss:248.925	eval-q0.9_loss:367.624
[140]	train-rmse:476.185	eval-rmse:508.911	train-q0.9_loss:238.952	eval-q0.9_loss:354.782
[141]	train-rmse:466.857	eval-rmse:494.77	train-q0.9_loss:229.294	eval-q0.9_loss:342.242
[142]	train-rmse:457.831	eval-rmse:481.128	train-q0.9_loss:219.851	eval-q0.9_loss:329.509
[143]	train-rmse:449.042	eval-rmse:467.631	train-q0.9_loss:210.752	eval-q0.9_loss:317.758
[144]	train-rmse:440.635	eval-rmse:453.839	train-q0.9_loss:201.984	eval-q0.9_loss:306.12
[145]	train-rms

[226]	train-rmse:262.012	eval-rmse:235.444	train-q0.9_loss:27.2685	eval-q0.9_loss:56.7474
[227]	train-rmse:261.609	eval-rmse:235.369	train-q0.9_loss:27.1162	eval-q0.9_loss:56.8562
[228]	train-rmse:261.446	eval-rmse:235.722	train-q0.9_loss:27.0024	eval-q0.9_loss:57.1743
[229]	train-rmse:261.242	eval-rmse:235.616	train-q0.9_loss:26.8418	eval-q0.9_loss:57.0758
[230]	train-rmse:261.132	eval-rmse:236.124	train-q0.9_loss:26.6893	eval-q0.9_loss:57.2494
[231]	train-rmse:261.316	eval-rmse:234.201	train-q0.9_loss:26.5392	eval-q0.9_loss:57.7299
[232]	train-rmse:260.8	eval-rmse:233.867	train-q0.9_loss:26.4108	eval-q0.9_loss:57.8884
[233]	train-rmse:260.864	eval-rmse:235.731	train-q0.9_loss:26.3239	eval-q0.9_loss:57.1402
[234]	train-rmse:260.581	eval-rmse:235.695	train-q0.9_loss:26.2558	eval-q0.9_loss:57.1233
[235]	train-rmse:260.451	eval-rmse:236.003	train-q0.9_loss:26.205	eval-q0.9_loss:57.1264
[236]	train-rmse:259.547	eval-rmse:235.279	train-q0.9_loss:26.0165	eval-q0.9_loss:57.377
[237]	train-rm

[318]	train-rmse:229.468	eval-rmse:231.209	train-q0.9_loss:21.1989	eval-q0.9_loss:60.3754
[319]	train-rmse:228.664	eval-rmse:230.515	train-q0.9_loss:21.1599	eval-q0.9_loss:60.638
[320]	train-rmse:229.341	eval-rmse:231.297	train-q0.9_loss:21.1571	eval-q0.9_loss:60.3756
[321]	train-rmse:229.231	eval-rmse:230.874	train-q0.9_loss:21.1551	eval-q0.9_loss:60.3951
[322]	train-rmse:229.319	eval-rmse:231.364	train-q0.9_loss:21.1535	eval-q0.9_loss:60.3408
[323]	train-rmse:229.042	eval-rmse:231.164	train-q0.9_loss:21.1379	eval-q0.9_loss:60.7392
[324]	train-rmse:228.891	eval-rmse:231.583	train-q0.9_loss:21.1022	eval-q0.9_loss:60.5164
[325]	train-rmse:228.254	eval-rmse:230.853	train-q0.9_loss:21.0775	eval-q0.9_loss:60.8861
[326]	train-rmse:228.424	eval-rmse:231.08	train-q0.9_loss:21.0314	eval-q0.9_loss:60.7063
[327]	train-rmse:227.64	eval-rmse:230.983	train-q0.9_loss:20.867	eval-q0.9_loss:61.2038
[328]	train-rmse:226.913	eval-rmse:230.966	train-q0.9_loss:20.8209	eval-q0.9_loss:61.4619
[329]	train-rm